<a href="https://colab.research.google.com/github/DariaK2/Computational-Linguistics-2025-26/blob/main/kovalenko%22vectors_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Bag-of-Words

In [ ]:
!pip install gensim

In [ ]:
import gensim
import gensim.downloader as api
from gensim import corpora
from gensim.models import TfidfModel
import numpy as np
from pprint import pprint

In [ ]:
# Данные для примера
documents = [
    "кошка сидит на ковре",
    "собака бегает по двору",
    "кошка играет с собакой",
    "птица летит высоко в небе",
    "собака и кошка дружат"
]

print("Исходные документы:")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc}")

In [ ]:
# Токенизация
tokenized_docs = [doc.lower().split() for doc in documents]
print("Токенизированные документы:")
pprint(tokenized_docs)

In [ ]:
# Создание словаря
dictionary = corpora.Dictionary(tokenized_docs)
print(f"Словарь: {dictionary.token2id}")

In [ ]:
dictionary.token2id

In [ ]:
# Создание Bag-of-Words представления
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
print("Bag-of-Words векторы (индекс_слова: частота):")
for i, doc_vec in enumerate(bow_corpus, 1):
    print(f"Документ {i}: {doc_vec}")

```
[['кошка', 'сидит', 'на', 'ковре'],
 ['собака', 'бегает', 'по', 'двору'],
 ['кошка', 'играет', 'с', 'собакой'],
 ['птица', 'летит', 'высоко', 'в', 'небе'],
 ['собака', 'и', 'кошка', 'дружат']]
 ```

```
Словарь: {'ковре': 0, 'кошка': 1, 'на': 2, 'сидит': 3, 'бегает': 4, 'двору': 5, 'по': 6, 'собака': 7, 'играет': 8, 'с': 9, 'собакой': 10, 'в': 11, 'высоко': 12, 'летит': 13, 'небе': 14, 'птица': 15, 'дружат': 16, 'и': 17}
```

In [ ]:
# Преобразование в плотные векторы для визуализации
def bow_to_dense(bow_corpus, dictionary):
    dense_vectors = []
    for doc in bow_corpus:
        dense_vec = [0] * len(dictionary)
        for idx, freq in doc:
            dense_vec[idx] = freq
        dense_vectors.append(dense_vec)
    return dense_vectors

dense_vectors = bow_to_dense(bow_corpus, dictionary)
print("Плотные векторы Bag-of-Words:")
print("Слова:", list(dictionary.token2id.keys()))
for i, vec in enumerate(dense_vectors, 1):
    print(f"Док {i}: {vec}")

```
Документ 1: [(0, 1), (1, 1), (2, 1), (3, 1)]
Документ 2: [(4, 1), (5, 1), (6, 1), (7, 1)]
Документ 3: [(1, 1), (8, 1), (9, 1), (10, 1)]
Документ 4: [(11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
Документ 5: [(1, 1), (7, 1), (16, 1), (17, 1)]
```

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Визуализация
matrix = np.array(dense_vectors)
words = list(dictionary.token2id.keys())
doc_names = [f"Doc {i+1}" for i in range(len(documents))]

print(f"Matrix shape: {matrix.shape}")

plt.figure(figsize=(25, 6))

plt.subplot(1, 2, 1)
sns.heatmap(matrix,
            annot=True,
            fmt='d',
            xticklabels=words,
            yticklabels=doc_names,
            cmap='Blues',
            cbar_kws={'label': 'Word Frequency'})
plt.title('Bag-of-Words Matrix Heatmap')
plt.xticks(rotation=45)
plt.ylabel('Documents')
plt.xlabel('Words')


### TF-IDF

In [ ]:
tfidf_model = TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

print("TF-IDF векторы (индекс_слова: tfidf_вес):")
for i, doc_vec in enumerate(tfidf_corpus, 1):
    print(f"Документ {i}: {[(dictionary[idx], round(score, 3)) for idx, score in doc_vec]}")

In [ ]:
# Преобразуем TF-IDF в плотную матрицу для визуализации
def tfidf_to_dense(tfidf_corpus, dictionary):
    dense_vectors = []
    for doc in tfidf_corpus:
        dense_vec = [0.0] * len(dictionary)
        for idx, score in doc:
            dense_vec[idx] = score
        dense_vectors.append(dense_vec)
    return dense_vectors

tfidf_dense = tfidf_to_dense(tfidf_corpus, dictionary)
tfidf_matrix = np.array(tfidf_dense)
words = list(dictionary.token2id.keys())
doc_names = [f"Док {i+1}" for i in range(len(tfidf_corpus))]

plt.figure(figsize=(40, 20))

plt.subplot(2, 3, 1)
sns.heatmap(tfidf_matrix,
            annot=True,
            fmt='.3f',
            xticklabels=words,
            yticklabels=doc_names,
            cmap='YlOrRd',
            cbar_kws={'label': 'TF-IDF Вес'})
plt.xticks(rotation=45)

plt.subplot(2, 3, 2)
# Получаем исходные TF значения из bow_corpus
tf_dense = bow_to_dense(bow_corpus, dictionary)
tf_matrix = np.array(tf_dense)

# Сравниваем TF и TF-IDF для первого документа
doc_idx = 0
x_pos = np.arange(len(words))
width = 0.35

plt.bar(x_pos - width/2, tf_matrix[doc_idx], width, label='TF', alpha=0.7, color='blue')
plt.bar(x_pos + width/2, tfidf_matrix[doc_idx], width, label='TF-IDF', alpha=0.7, color='red')
plt.xticks(x_pos, words, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

#### **1. TF (Term Frequency) - Частота термина**
```python
TF(t,d) = count(t,d) / total_terms(d)
```
или упрощенно:
```python
TF(t,d) = count(t,d)  # просто количество вхождений
```

#### **2. IDF (Inverse Document Frequency) - Обратная частота документа**
```python
IDF(t) = log( N / (DF(t) + 1) ) + 1
```
Где:
- **N** - общее количество документов
- **DF(t)** - количество документов, содержащих слово t
- **+1** в знаменателе - сглаживание, чтобы не было "околонулевых" значений (smoothing)
- **+1** в конце - для избежания нулевых весов

#### **3. TF-IDF - итоговый вес**
```python
TF-IDF(t,d) = TF(t,d) × IDF(t)
```

**Сравнение разных формул IDF**

| Формула | Редкое слово (DF=1) | Частое слово (DF=100) | Сверхчастое (DF=990) |
|---------|---------------------|----------------------|---------------------|
| `log(N/DF)` | 6.91 | 2.30 | 0.01 |
| `log(N/(DF+1))` | 6.90 | 2.29 | 0.009 |
| `log(N/(DF+1)) + 1` | **7.90** | **3.29** | **1.009** |


### Word2Vec

In [ ]:
# Используем небольшую модель для демонстрации
print("Загрузка предобученной модели Word2Vec...")
w2v_model = api.load("glove-wiki-gigaword-50")  # Маленькая модель

# Функция для получения эмбеддинга документа
def get_doc_embedding(tokens, model):
    word_vectors = []
    for token in tokens:
        try:
            word_vectors.append(model[token])
        except KeyError:
            continue
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Получаем эмбеддинги для документов
doc_embeddings = []
for i, tokens in enumerate(tokenized_docs):
    embedding = get_doc_embedding(tokens, w2v_model)
    doc_embeddings.append(embedding)
    print(f"\nДокумент {i+1}: {documents[i]}")
    print(f"Эмбеддинг (первые 10 значений): {embedding[:10].round(4)}")
    print(f"Размер: {embedding.shape}")

# Косинусная схожесть между документами
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(doc_embeddings)
print("\nМатрица косинусной схожести:")
print("     Д1    Д2    Д3    Д4    Д5")
for i, row in enumerate(similarity_matrix):
    print(f"Д{i+1}  " + "  ".join([f"{x:.3f}" for x in row]))

*Как работает Word2Vec мы разберем на следующей лекции..*

### Домашка

1. Скачать [датасет по ссылке](https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/refs/heads/main/data/english/test.csv) (просто запустить строку ниже)

In [ ]:
!wget https://raw.githubusercontent.com/tyqiangz/multilingual-sentiment-datasets/refs/heads/main/data/english/test.csv

2. Открыть csv и сохранить текст в переменную (макс. балл == 1)

In [ ]:
import pandas as pd

df = pd.read_csv('test.csv')

print(df.head())
print(df.columns)

texts = df['text'].tolist()
print(f"Всего текстов: {len(texts)}")
print("Пример:", texts[0])

3. Извлечь столбец с текстами (макс. балл == 1)

In [ ]:
text_column = df['text']
print(text_column.head())

4. Построить Bag-of-Words (макс. балл == 3)

In [ ]:
# Задача 1.1: Создайте словарь и BoW представление (макс. балл == 2)
# - Приведите тексты к нижнему регистру
# - Удалите знаки препинания
# - Постройте словарь
# - Преобразуйте документы в BoW векторы

import string
from gensim import corpora

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.split()

tokenized_texts = [preprocess(t) for t in texts]
dictionary = corpora.Dictionary(tokenized_texts)
print(f"Всего уникальных слов: {len(dictionary)}")
bow_corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_texts]

# Задача 1.2: Визуализируйте результаты (макс. балл == 1)
# - Создайте таблицу частот слов
# - Постройте heatmap матрицы BoW
# - Найдите самые частые слова в корпусе

def bow_to_dense(bow_corpus, dictionary):
    dense = []
    for doc in bow_corpus:
        vec = [0] * len(dictionary)
        for word_id, freq in doc:
            vec[word_id] = freq
        dense.append(vec)
    return dense

bow_dense = bow_to_dense(bow_corpus, dictionary)
bow_matrix = np.array(bow_dense)

word_freq = np.sum(bow_matrix, axis=0)
freq_df = pd.DataFrame({
    'word': [dictionary[i] for i in range(len(dictionary))],
    'frequency': word_freq
}).sort_values('frequency', ascending=False)

print("Топ-10 слов:")
print(freq_df.head(10))

import matplotlib.pyplot as plt
import seaborn as sns

top_words = freq_df.head(20)['word'].tolist()
top_indices = [dictionary.token2id[w] for w in top_words]
bow_small = bow_matrix[:, top_indices]

plt.figure(figsize=(12, 8))
sns.heatmap(bow_small[:50],  # только первые 50 документов
            xticklabels=top_words,
            cmap='Blues',
            annot=False)
plt.title('BoW Heatmap (топ-20 слов, первые 50 доков)')
plt.xlabel('Слова')
plt.ylabel('Документы')
plt.xticks(rotation=45)
plt.show()

5. Построить TF-IDF (макс. балл == 5)

In [ ]:
# Задача 2.1: Примените TF-IDF к BoW представлению (макс. балл == 2)
# - Используйте TfidfModel из gensim
# - Получите TF-IDF векторы для каждого документа

from gensim.models import TfidfModel

tfidf_model = TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

def tfidf_to_dense(tfidf_corpus, dictionary):
    dense = []
    for doc in tfidf_corpus:
        vec = [0.0] * len(dictionary)
        for idx, score in doc:
            vec[idx] = score
        dense.append(vec)
    return dense

tfidf_dense = tfidf_to_dense(tfidf_corpus, dictionary)
tfidf_matrix = np.array(tfidf_dense)

# Задача 2.2: Проанализируйте веса TF-IDF (макс. балл == 3)
# - Для каждого слова вычислите: (макс. балл == 2)
#   * TF (term frequency) в каждом документе
#   * DF (document frequency) во всем корпусе
#   * IDF (inverse document frequency)
#   * значение TF-IDF
# - Сохраните результат в *.сsv (макс. балл == 1)
# - Прикрепите *.csv в ваш репозиторий

import numpy as np

def analyze_tfidf_components(bow_corpus, dictionary):
  tf_matrix = np.zeros((len(bow_corpus), len(dictionary)))
  for doc_idx, doc in enumerate(bow_corpus):
    for word_idx, freq in doc:
      tf_matrix[doc_idx, word_idx] = freq

  df_vector = np.sum(tf_matrix > 0, axis=0)
  N = len(bow_corpus)
  idf_vector = np.log(N / (df_vector + 1)) + 1

  tfidf_manual = tf_matrix * idf_vector

  return tf_matrix, df_vector, idf_vector, tfidf_manual

tf_matrix, df_vector, idf_vector, tfidf_manual = analyze_tfidf_components(bow_corpus, dictionary)

words = ['good', 'bad', 'not', 'movie']
words = [w for w in words if w in dictionary.token2id]
print("Будем анализировать:", words)

rows = []

for word in words:
    word_id = dictionary.token2id[word]

    for doc_idx in range(len(bow_corpus)):
        tf = tf_matrix[doc_idx, word_id]        # сколько раз в документе
        if tf == 0:
            continue  # пропускаем, где слова нет

        df = int(df_vector[word_id])            # в скольки доках
        idf = round(idf_vector[word_id], 4)
        tfidf_val = round(tfidf_manual[doc_idx, word_id], 4)

        rows.append({
            'word': word,
            'document': doc_idx + 1,
            'TF': int(tf),
            'DF': df,
            'IDF': idf,
            'TF-IDF': tfidf_val
        })

import csv

with open('tfidf_analysis.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['word', 'document', 'TF', 'DF', 'IDF', 'TF-IDF'])
    for row in rows:
        writer.writerow([row['word'], row['document'], row['TF'], row['DF'], row['IDF'], row['TF-IDF']])

print("Сохранено в tfidf_analysis.csv")



##### **Как вычислить компоненты вручную**

```python
def analyze_tfidf_components(bow_corpus, dictionary):
    # Шаг 1: Создаем матрицу TF (term frequency)
    tf_matrix = np.zeros((len(bow_corpus), len(dictionary)))
    for doc_idx, doc in enumerate(bow_corpus):
        for word_idx, freq in doc:
            tf_matrix[doc_idx, word_idx] = freq
    
    # Шаг 2: Вычисляем DF (document frequency)
    df_vector = np.sum(tf_matrix > 0, axis=0)  # Количество документов с каждым словом
    
    # Шаг 3: Вычисляем IDF (inverse document frequency)
    N = len(bow_corpus)  # общее количество документов
    idf_vector = np.log(N / (df_vector + 1)) + 1
    
    # Шаг 4: Вычисляем TF-IDF вручную
    tfidf_manual = tf_matrix * idf_vector
    
    return tf_matrix, df_vector, idf_vector, tfidf_manual
```

6. Произвести сравните

*Эта часть не оценивается, но будет давать доп. баллы на экзамене для тех, кто выполнит её*

In [ ]:
# Сравните BoW и TF-IDF для конкретных слов
# - Выберите 3-4 слова из датасета (выберите слова, характерные для датасета)
# - Постройте графики сравнения их весов в BoW и TF-IDF

def bow_to_dense(bow_corpus, dictionary):
    dense = []
    for doc in bow_corpus:
        vec = [0] * len(dictionary)
        for idx, freq in doc:
            vec[idx] = freq
        dense.append(vec)
    return dense

bow_dense = bow_to_dense(bow_corpus, dictionary)
bow_matrix = np.array(bow_dense)

tfidf_dense = tfidf_to_dense(tfidf_corpus, dictionary)
tfidf_matrix = np.array(tfidf_dense)

words = ['call', 'free', 'txt', 'win']
words = [w for w in words if w in dictionary.token2id]
print("Графики для слов:", words)

import matplotlib.pyplot as plt

def compare_bow_tfidf(words, bow_dense, tfidf_dense, dictionary):
    n = len(words)

    # Правильная сетка
    if n <= 2:
        fig, axes = plt.subplots(1, n, figsize=(7 * n, 6))
        if n == 1:
            axes = [axes]
        else:
            axes = list(axes)
    else:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.flatten()[:n]  # обрезаем лишние, если слов < 4

    for i, word in enumerate(words):
        word_id = dictionary.token2id[word]

        bow_weights = [doc[word_id] for doc in bow_dense]
        tfidf_weights = [doc[word_id] for doc in tfidf_dense]
        x = range(len(bow_dense))

        ax = axes[i]
        ax.bar([p - 0.2 for p in x], bow_weights, width=0.4, label='BoW', alpha=0.7, color='skyblue')
        ax.bar([p + 0.2 for p in x], tfidf_weights, width=0.4, label='TF-IDF', alpha=0.7, color='salmon')

        ax.set_title(f'"{word}"', fontsize=13)
        ax.set_xlabel('Документы')
        ax.set_ylabel('Вес')
        ax.legend()

        # Подписи — только если документов мало
        if len(x) <= 30:
            step = max(1, len(x) // 10)
            ax.set_xticks(x[::step])
            ax.set_xticklabels([f'D{i+1}' for i in x[::step]], rotation=45)
        else:
            ax.set_xticks([])

    plt.suptitle('Сравнение BoW и TF-IDF', fontsize=16)
    plt.tight_layout()
    plt.show()

compare_bow_tfidf(words, bow_dense, tfidf_dense, dictionary)

##### **Как построить сравнительный график**
```python
def compare_bow_tfidf(words, bow_dense, tfidf_dense, dictionary):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    for i, word in enumerate(words):
        row, col = i // 2, i % 2
        word_id = dictionary.token2id[word]
        
        # Веса для этого слова во всех документах
        bow_weights = [doc[word_id] for doc in bow_dense]
        tfidf_weights = [doc[word_id] for doc in tfidf_dense]
        
        x = range(len(bow_dense))
        axes[row, col].bar(x, bow_weights, alpha=0.7, label='BoW', width=0.4)
        axes[row, col].bar([p + 0.4 for p in x], tfidf_weights, alpha=0.7, label='TF-IDF', width=0.4)
        axes[row, col].set_title(f'Сравнение весов: "{word}"')
        axes[row, col].legend()
        axes[row, col].set_xticks([p + 0.2 for p in x])
        axes[row, col].set_xticklabels([f'Док {i+1}' for i in x])
    
    plt.tight_layout()
    plt.show()
```